In [25]:
from pandas import DataFrame, read_csv
from os.path import join, basename
from os.path import splitext, join
from DNASkittleUtils.Annotations import parseGFF
from DNASkittleUtils.Contigs import read_contigs
from DNASkittleUtils.CommandLineUtils import just_the_name
from DNASkittleUtils.DDVUtils import pp
filename = r"D:\josiah\Documents\Research\Arabidopsis\TAIR10_GFF3_genes_only.gff3"

In [2]:
df = read_csv(r"D:\josiah\Documents\Research\Arabidopsis\functional_annotation.csv",sep='\t', error_bad_lines=False)# fields='Gene_Family	Gene_Name	Genomic_Locus(BAC)	Genomic_Locus_Tag	Protein_Function'.split())

In [9]:
df.head()

,Gene_Family,Gene_Name,Genomic_Locus(BAC),Genomic_Locus_Tag,Protein_Function
0,IQD Protein Family,IQD1,NaN,At3g09710,NaN
1,IQD Protein Family,IQD2,NaN,At5g03040,NaN
2,IQD Protein Family,IQD3,NaN,At3g52290,NaN
3,IQD Protein Family,IQD4,NaN,At2g26410,NaN
4,IQD Protein Family,IQD5,NaN,At3g22190,NaN


In [107]:
len(df)

8693

In [106]:
sum(df['Gene_Family'].notnull())

8693

In [22]:
sum(df['Protein_Function'].str.contains('putative')==False)

3723

In [26]:
sum(df['Protein_Function'].notnull() & (df['Protein_Function'].str.contains('hypothetical protein')==False))

5804

In [25]:
6112 /28775

0.21240660295395308

In [36]:
8472 /28775

0.2944222415291051

In [46]:
len(set(tags))

7647

In [47]:
len(set([str(n) for n in tags if n]))

7647

In [51]:
6805/28775

0.2364900086880973

In [48]:
len(set([str(n).upper() for n in tags if n]))

6805

# Filter out to only genes with mentions in functional_annotation.csv

In [45]:
tag = df['Genomic_Locus_Tag'].notna()
tags = df[tag]['Genomic_Locus_Tag']
sum(tag), len(tag)

(8472, 8693)

In [55]:
upper_case_loci = set([str(n).upper() for n in tags])
len(upper_case_loci)

6805

In [86]:

filtered_path = splitext(filename)[0] + '_with_functions' + splitext(filename)[1]
with open(filename) as file:
    with open(filtered_path, 'w') as outfile:
        for line in file.readlines():
            attributes = line.split('\t')[8].split(';')
            locus = attributes[0][3:]
            if locus.upper() in upper_case_loci or name.upper() in upper_case_loci:
                outfile.write(line)

6652 lines

#### Create Gene function Lookup

In [116]:
gene_function = {}
for index, row in df[tag].iterrows():
    try: 
        gene = row['Genomic_Locus_Tag'].upper()
    except AttributeError:
        print(row['Genomic_Locus_Tag'])
    function = row['Protein_Function'] if str(row['Protein_Function']) != 'nan' else row['Gene_Family']
    forbidden = ['\uffa0', 'basic', 'hypothetical protein', 'unknown protein', 'putative', 'hypothetical  protein']
    for bad in forbidden:
        function = function.replace(bad, '')
    if not function.strip() or str(function) == 'nan':
        function = row['Gene_Family'].replace('basic', '')
    gene_function[gene] = function
#     break
gene_function

{'AT3G47090': 'receptor kinase-like protein',
 'AT1G27400': 'L17 in small subunit',
 'AT3G19553': 'amino acid permease, ',
 'AT1G78600': 'C2C2-CO-like Transcription Factor Family',
 'AT5G55360': 'wax synthase-like protein',
 'AT5G08130': 'bHLH Transcription Factor Family',
 'AT3G11940': 'S5 in small subunit',
 'AT1G28370': 'AP2-EREBP Transcription Factor Family',
 'AT1G62660': 'beta-fructosidase',
 'AT4G19210': 'RNase L inhibitor-like protein',
 'AT4G37260': 'MYB Transcription Factor Family',
 'AT3G13580': 'L7 in small subunit',
 'AT4G34030': '3-methylcrotonyl-CoA carboxylase non-biotinylated subunit (MCCB)',
 'AT2G46950': ' cytochrome P450',
 'AT3G27970': 'C2H2 Transcription Factor Family',
 'AT2G45110': 'beta-expansin pollen allergen protein',
 'AT4G24750': ' protein',
 'AT4G37640': 'plasma membrane-type calcium ATPase (ACA2)',
 'AT1G56680': 'chitinase, ',
 'AT3G09630': 'L4 in small subunit',
 'AT5G15200': 'S9 in small subunit',
 'AT1G48490': 'protein kinase, similar to IRE',
 'AT5G4

** ----------------- TODO: Come back and make the name pretty ---------------------**

### Decorate Annotation with functions

In [117]:
with open(filtered_path, 'r') as genes:
    with open(r"D:\josiah\Documents\Research\Arabidopsis\TAIR10_annotated_genes.gff", 'w') as final_annotation:
        for line in genes.readlines():
            parts = line.split('\t')
            locus = parts[8].split(';')[0][3:]
            function = gene_function[locus]
            if locus in gene_function:
                final_annotation.write('\t'.join(parts[:8] + ['Name=' + function]) +'\n')

In [113]:
df[df['Genomic_Locus_Tag'] =='At1g01570']

,Gene_Family,Gene_Name,Genomic_Locus(BAC),Genomic_Locus_Tag,Protein_Function
4611,Glycosyltransferase Gene Families,NaN,NaN,At1g01570,hypothetical protein


In [118]:
'AT1G01020' in upper_case_loci

False

# Repeat the Process for ATH_GO_SLIM

In [2]:
go = read_csv(r"D:\josiah\Documents\Research\Arabidopsis\ATH_GO_GOSLIM.txt",sep='\t', error_bad_lines=False,
             usecols=['locus name', 'GO term','GOslim term', 'Date annotated'])
go.head()

,locus name,GO term,GOslim term,Date annotated
0,AT1G01010,"regulation of transcription, DNA-templated",other cellular processes,2018-04-02
1,AT1G01010,"regulation of transcription, DNA-templated","transcription,DNA-dependent",2018-04-02
2,AT1G01010,DNA-binding transcription factor activity,transcription factor activity,2003-06-06
3,AT1G01010,"regulation of transcription, DNA-templated",other cellular processes,2018-09-12
4,AT1G01010,integral component of membrane,other membranes,2018-09-04


In [3]:
sum(go['GO term'].notnull() & go['locus name'].notnull()) ,len(go)

(349103, 349103)

Terrific!  No missing data

In [4]:
go_upper = {cell.upper() for cell in go['locus name']}
len(go_upper)

30828

30,828 is slightly more genes than TAIR10 annotation...  because there are non-coding regions...

In [5]:
go_function = {}
for index, row in go.iterrows():
    gene = row['locus name']
    function = row['GO term']
    forbidden = ['biological_process', 'cellular_component', 'molecular_function',  'extracellular region', 'nucleus',]
    #            'mitochondrion',  'cytoplasm', 'cytosol', 'chloroplast', 'membrane', 'plasma membrane']

    if function not in forbidden:
        if gene not in go_function or len(go_function[gene]) < len(function):  # use the longest
            go_function[gene] = function
len(go_function)

25302

In [7]:
functional_gff = r"D:\josiah\Documents\Research\Arabidopsis\GO_annotation.gff"
list(go_function.values())[:100]

['integral component of membrane',
 'copper ion transport',
 'mitochondrion',
 'aspartate family amino acid biosynthetic process',
 'singlet oxygen-mediated programmed cell death',
 'protein serine/threonine phosphatase activity',
 'tRNA (guanine-N2-)-methyltransferase activity',
 'regulation of abscisic acid-activated signaling pathway',
 'phosphatidylinositol-3,4,5-trisphosphate 5-phosphatase activity',
 'triplet codon-amino acid adaptor activity',
 'integral component of membrane',
 'chloroplast',
 'response to aluminum ion',
 'mitochondrion',
 'mitochondrion',
 'triplet codon-amino acid adaptor activity',
 'cytoplasm',
 'sn-1-glycerol-3-phosphate C16:0-DCA-CoA acyl transferase activity',
 'endoplasmic reticulum-Golgi intermediate compartment membrane',
 'defense response to bacterium, incompatible interaction',
 'triplet codon-amino acid adaptor activity',
 'pyruvate dehydrogenase (acetyl-transferring) activity',
 'regulation of transcription, DNA-templated',
 'mitochondrion',
 'tr


    with open(filename, 'r') as genes:
        with open(functional_gff, 'w') as final_annotation:
            for line in genes.readlines():
                parts = line.split('\t')
                locus = parts[8].split(';')[0][3:]
                if locus in go_function:
                    function = go_function[locus]
                    final_annotation.write('\t'.join(parts[:8] + ['Name=%s;ID=%s'% (function, locus) ]) +'\n')

22,103 genes

In [10]:
from DNASkittleUtils.Contigs import Contig, read_contigs, write_contigs_to_file

In [16]:
chromosomes = read_contigs(r"D:\josiah\Documents\Research\Arabidopsis\TAIR10_chr_all.fas")
for chrm in chromosomes:
    chrm.name = chrm.name.split()[0]
    write_contigs_to_file(join(r"D:\josiah\Documents\Research\Arabidopsis\Chromosomes", chrm.name + '.fa'), [chrm])

Done writing  1 contigs and 30,427,671bp
Done writing  1 contigs and 19,698,289bp
Done writing  1 contigs and 23,459,830bp
Done writing  1 contigs and 18,585,056bp
Done writing  1 contigs and 26,975,502bp
Done writing  1 contigs and 366,924bp
Done writing  1 contigs and 154,478bp


In [11]:
from functools import reduce
reduce(int.__mul__,[3,3,3,3,3,15]) + 200

3845

In [12]:
reduce(int.__mul__, [5,3,3,3,3,]) # coil_height

405

# Dice up chromosomes according to annotation
* Annotations will sometimes overlap. Sort by start position, grab nucleotides 
* and then look for next annotation whose end position is greater than nucleotides already taken
* Start next annotation then and repeat
* Some annotations will be lost completely, but not many if overlap is low, more likely case is truncation
* When no active annotations, mark with the same name: Chromosome X between genes ###
------------


In [13]:
annotation = parseGFF(functional_gff)

In [14]:
# force sorting
for chr_entries in annotation.values():
    chr_entries.sort(key=lambda e: e.start)

In [15]:
all([entry.start >= annotation['2'][i-1].start for i, entry in enumerate(annotation['2'][:]) if i])

True

In [41]:
def write_functional_gene_contigs_from_chromosome(chr_seq_file):
    #assume the order of annotations by start, create sequential contigs
    nucleotides_claimed = set()
    annotated_sequences = []  # in order from start position, end position never overlaps next start
    chr_seq = read_contigs(chr_seq_file)[0].seq
    scaff_name = just_the_name(chr_seq_file)
    current_start = 0
    if scaff_name in annotation.keys():
        for entry in annotation[scaff_name]:
            new_start = entry.start - 1
            end = entry.end - 1
            if new_start + 1 > current_start:
                annotated_sequences.append(Contig('intergenic', chr_seq[current_start: new_start]))
                #print('SPacer', new_start-current_start)
            #else:
                #pass
            annotated_sequences.append(Contig(entry.name(), chr_seq[max(current_start, new_start) : end]))
            #print("Gene %i - %i" %(new_start, entry.end))
            current_start = end

    new_file = r"D:\josiah\Documents\Research\Arabidopsis\Chromosomes\chr"+ scaff_name + '_functions.fa'
    write_contigs_to_file(new_file, annotated_sequences)
    return [len(s.seq) for s in annotated_sequences], len(chr_seq) 

for chr_number in range(1,6):
    chr_seq_file = r"D:\josiah\Documents\Research\Arabidopsis\Chromosomes\%i.fa" % chr_number
    lengths, chr_length = write_functional_gene_contigs_from_chromosome(chr_seq_file)
    print(pp(sum(lengths)), 'Difference', pp(chr_length - sum(lengths)))
    

Done writing  12386 contigs and 30,453,674bp
30,453,674 Difference -26,003
Done writing  7337 contigs and 19,731,769bp
19,731,769 Difference -33,480
Done writing  9174 contigs and 23,479,378bp
23,479,378 Difference -19,548
Done writing  7061 contigs and 18,611,740bp
18,611,740 Difference -26,684
Done writing  10787 contigs and 27,009,486bp
27,009,486 Difference -33,984


In [ ]:
19731769

In [31]:
len(lengths)

12386

### Layout
* Ideogram has zero padding between contigs
* Mouseover will iterate contigs til it finds the right one (could technically be overlapping)
* 